# Knowledge Error Detection using K-means and Adaptive Clustering

## 1. Clustering positive entity vectors using k-means
- Attempt to k-means clustering with an entity vector of triples consisting of <entity, type, person>.
- A positive entity means that the entity is a person in the triple of <entity, type, person>.
    - (e.g. entity : 'Biden', 'Trump', ...)
- Find the maximum Euclidean distance between the centroid of each clusters and the elements included in the cluster.
- The maximum Euclidean distance obtained from each cluster means the radius $r$ of the cluster, and entities located between the radius from the centroid are classified as entities of the cluster.

## 2. Apply the adaptive clustering method.
- Add the negative entity vector to the vector space clustered by the positive entity vector.
- A negative entity means that the entity is not a person in the triple of <entity, type, person>.
    - (e.g. entity : 'Titanic'(Film), 'New York'(City), ...)
- Negative vectors are included in the cluster by calculating the distance between each centroid of the cluster.
- Find the optimal $\delta $(0.6 ~ 1.0) that will maximize the $f1-score$ of each cluster.
- The $\delta $ is multiplied by the cluster radius $r$ to create a new cluster range $r'$.
    - $f1-score = \frac{2\times precision\times recall}{precision+recall}$
    - $precision = \frac{TP}{TP+FP}$
    - $recall = \frac{TP}{TP+FN}$
    - $TP(True Positive)$ = Number of positive entities included in the cluster  
    - $FP(False Positive)$ = Number of negative entities included in the cluster
    - $FN(False Negative)$ = The number of positive entities included in the cluster before the cluster range was adjusted but not included after the adjustment.  

## 3. Perform error triple detection.
- Add the positive and negative vectors of the test data to the vector space.
- Apply a new radius $r'$ to each cluster multiplied by the optimal $\delta $.
- Entities included in the cluster are classified as positive entities, and entities not included in any cluster are classified as negative entities.

In [1]:
# custom functions
from AdaClustering.ClusteringAlgorithm import kmeans_alg, grant_to_cluster
from evaluation.metrics import get_initial_TP, get_ada_matrics, get_optimal_matrics
from util.counter import count_element, count_TPFP

import numpy as np
import pandas as pd
import glob ,os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances  #k-means using euclidean_distances
import gensim

## Load skip-gram embedding vectors from model
- dataname options :
    - dbpedia / freebase / wisekb

In [2]:
#data_name options = freebase, dbpedia, wisekb
dataname = 'freebase'
k_dict = {'freebase' : 21, 'dbpedia' : 21, 'wiseKB' : 27}
opt_k = k_dict[dataname]

save_model_path = f'./data/SkipgramEntityVectors/skipgram_{dataname}_person'

"""
Word2VecKeyedVector object
- key(string) : word
- value(numpy array) : word vector
"""
keyedvectors = gensim.models.KeyedVectors.load_word2vec_format(save_model_path) 

### load entity label from file

In [3]:
if dataname=='freebase':
    sep = '\t'
else:
    sep = ' '
    
path = f'./data/{dataname}/'
all_files = glob.glob(os.path.join(path, "*.txt"))    
filename_list = ['train_positive', 'train_negative', 'test_positive', 'test_negative']
train_pos_entities = []
train_neg_entities = []
test_pos_entities = []
test_neg_entities = []
all_entities = []
for file_name in filename_list:
    for directory in all_files:
        if file_name in directory:
            entities = []
            data_name = file_name.split('/')[0]
            with open(directory) as f:          
                for triple in f:
                    entities.append(triple.split(sep)[0].strip())
            print(f'{data_name} contains {len(entities)} entities')
            print(entities[:5], '\n')
            all_entities.append(entities)
train_pos_entities, train_neg_entities, test_pos_entities, test_neg_entities = all_entities

train_positive contains 20000 entities
['23899570', '49325820', '59159370', '49921652', '39747070'] 

train_negative contains 5000 entities
['23490277', '58522601', '35716929', '55590744', '43009747'] 

test_positive contains 5000 entities
['45555043', '45834103', '22467328', '58221651', '19409672'] 

test_negative contains 5000 entities
['2507774', '5774863', '7240658', '57271678', '40535264'] 



## 1. Clustering positive entity vectors using k-means 

In [4]:
vectors, centroids, c_label = kmeans_alg(train_pos_entities, opt_k, keyedvectors)

# dictionary for labeling entity with cluster - key : cluster label / - value : list of vectors
dicts_each_cluster = {}
for c in range(opt_k):
    same_Cluster = []    
    for idx, label in enumerate(c_label):
        if label == c:
            same_Cluster.append(vectors[idx].tolist())    
    dicts_each_cluster[c]= list(same_Cluster)

# dictionary for each cluster's max Euclidean distance 
max_distance_each_cluster = {}

for i in dicts_each_cluster.keys():
    in_vectors = dicts_each_cluster[i]
    tmp_distance = euclidean_distances(in_vectors, centroids[i].reshape(1,-1))
    
    # max_distance_each_cluster - key : cluster lable / - value : max_distance
    max_distance_each_cluster[i] = max(tmp_distance)[0].astype(np.float32) 
    
# Get vectors from embedding model 
train_pos_vector = list(map(lambda x: (x, keyedvectors.word_vec(x)), train_pos_entities))
train_neg_vector = list(map(lambda x: (x, keyedvectors.word_vec(x)), train_neg_entities))

### Check Statistics before delta optimzing 

In [5]:
true_cluster = grant_to_cluster(train_pos_vector, 1, max_distance_each_cluster, centroids)
false_cluster = grant_to_cluster(train_neg_vector, 1, max_distance_each_cluster, centroids)
true_count, false_count = count_TPFP(centroids, true_cluster, false_cluster)

result_list = []

# 각 클러스터에 포함된 positive 엔티티, negative 엔티티수로 각 클러스터에 대한 precision 지표 산출
for cluster in range(len(centroids)):
    TP = true_count[cluster] # True Positive : 클러스터 포함되어있는 positive 엔티티
    FP = false_count[cluster] # False Positive : 클러스터에 포함되어있는 negative 엔티티
    if (TP+FP) == 0:
        precision = 0.0
    else:
        precision = TP / (TP+FP)
    result_list.append([TP,FP,precision])

# first_result_df
pre_result_df = pd.DataFrame(result_list, columns=['initial_cluster_pos', 
                                                   'initial_cluster_neg', 
                                                   'initial_cluster_precision'])
pre_result_df.head()

,initial_cluster_pos,initial_cluster_neg,initial_cluster_precision
0,380,40,0.904762
1,1743,57,0.968333
2,897,271,0.767979
3,1808,64,0.965812
4,426,20,0.955157


## 2. Apply the adaptive clustering method.

### optimize delta for each clusters

In [6]:
P, R, F1, TP, FN, FP, initial_TP = get_ada_matrics(train_pos_vector, train_neg_vector, 
                                                   max_distance_each_cluster, centroids)


TP_list = TP[np.array(F1).argmax(0),list(range(TP.shape[1]))]

#cluster의 f1 score가 최대가 되는 지점의 delta를 optimal delta로 선정
optimal_deltas = np.array(F1).argmax(0)
optimal_deltas = (optimal_deltas * 0.01)+0.6

# positive entity수가 0인 cluster는 delta를 0으로 변환
optimal_deltas = np.array([d if tp != 0 else 0 for d, tp in zip(optimal_deltas, TP_list)])

print('optimaldeltas each clusters : \n', optimal_deltas)

optimaldeltas each clusters : 
 [0.82 0.68 0.73 0.68 0.93 0.68 0.81 0.74 0.82 0.68 0.68 0.85 0.9  0.93
 0.7  0.77 0.6  0.7  0.85 0.68 1.  ]


In [7]:
# 구한 optimal delta를 cluster에 적용하여 각 cluster에 대한 precision, recall, TP, FP를 산출
optimalP, optimalR, optimalTP, optimalFP = get_optimal_matrics(train_pos_vector, train_neg_vector, 
                                                            max_distance_each_cluster,centroids, 
                                                            optimal_deltas, initial_TP)

matrics = []
for m1, m2, m3, m4, m5 in zip(optimalP, optimalR,  optimalTP, optimalFP, optimal_deltas):
    matrics.append([m1, m2, m3, m4, m5])
    
opt_delta_df = pd.DataFrame(matrics, columns=['opt_precision', 'opt_recall', 'opt_TP','opt_FP', 'opt_delta'])
tmp_parse_df = opt_delta_df[['opt_delta','opt_TP','opt_FP', 'opt_precision']]
train_result_df = pd.concat([pre_result_df, tmp_parse_df], axis = 1)
train_result_df.head()

,initial_cluster_pos,initial_cluster_neg,initial_cluster_precision,opt_delta,opt_TP,opt_FP,opt_precision
0,380,40,0.904762,0.82,370,21,0.946292
1,1743,57,0.968333,0.68,1715,0,1.000000
2,897,271,0.767979,0.73,825,100,0.891892
3,1808,64,0.965812,0.68,1781,2,0.998878
4,426,20,0.955157,0.93,425,12,0.972540


## 3. Perform error triple detection.

In [8]:
# Get test vectors from embedding model 
test_pos_vector = list(map(lambda x: (x, keyedvectors.word_vec(x)), test_pos_entities))
test_neg_vector = list(map(lambda x: (x, keyedvectors.word_vec(x)), test_neg_entities))
initial_TP = get_initial_TP(test_pos_vector, max_distance_each_cluster, centroids)

# test 데이터의 embedding 벡터를 cluster에 할당하여 Precision, Recall 산출
optimalP, optimalR, optimalTP, optimalFP = get_optimal_matrics(test_pos_vector, test_neg_vector, 
                                                               max_distance_each_cluster, centroids, 
                                                               optimal_deltas, initial_TP)

In [9]:
matrics = []
for m1, m2, m3, m4, m5 in zip(optimalP, optimalR, optimalTP, optimalFP, optimal_deltas):
    matrics.append([m1, m2, m3, m4, m5])
    
output_df = pd.DataFrame(matrics, columns=['precision', 'recall', 'TP','FP', 'delta'])
tmp_output_df = output_df[['TP','FP', 'precision', 'recall']]
tmp_output_df.columns = ['Test_TP','Test_FP', 'Test_precision', 'Test_recall']

train_test_df = pd.concat([train_result_df, tmp_output_df], axis = 1)
train_test_df.head()

,initial_cluster_pos,initial_cluster_neg,initial_cluster_precision,opt_delta,opt_TP,opt_FP,opt_precision,Test_TP,Test_FP,Test_precision,Test_recall
0,380,40,0.904762,0.82,370,21,0.946292,101,20,0.834711,0.901786
1,1743,57,0.968333,0.68,1715,0,1.000000,362,47,0.885086,0.952632
2,897,271,0.767979,0.73,825,100,0.891892,227,146,0.608579,0.893701
3,1808,64,0.965812,0.68,1781,2,0.998878,454,101,0.818018,0.984816
4,426,20,0.955157,0.93,425,12,0.972540,112,21,0.842105,1.000000


In [10]:
# 테스트 데이터에 대한 error detection결과 및 precision, recall, F1 score 산출
test_TP = sum(train_test_df['Test_TP'].values)
test_FP = sum(train_test_df['Test_FP'].values)

test_precision = test_TP / (test_TP + test_FP)
test_recall = test_TP / len(test_pos_entities)
test_f1 = (2*test_precision*test_recall) / (test_precision+test_recall)

print(f'TP : {test_TP}')
print(f'FP : {test_FP}')
print(f'precision : {test_precision:.3f}')
print(f'recall : {test_recall:.3f}')
print(f'F1 : {test_f1:.3f}')

print(f'\nDetected {len(test_neg_entities) - test_FP} error triples')
print(f'Detected {len(test_pos_entities)-test_TP} normal triples as error triples')

TP : 4728
FP : 1508
precision : 0.758
recall : 0.946
F1 : 0.842

Detected 3492 error triples
Detected 272 normal triples as error triples
